In [1]:
import re
import pandas as pd

In [2]:
filename = "example-1.als"
lines = ""

In [3]:
with open(filename, "r") as f:
    lines = f.read()
print(lines)  

module alloy2cd

open util/boolean


abstract sig Vehicle   {
}

sig Car extends Vehicle {
	wheels: some Wheel,
	isdeflated: Bool
}

one sig Person {
	name: String,
	age: Int, 
	cars: some Car
}


sig Wheel  {
	cars: one Car
}

pred exp {
	 all p: Person | p.name in ("John" + "Paul" + "Michael")
} run exp for 3



In [4]:
# find package
module = re.findall('\s*module\s+(\w+)', lines)[0]

# find sigs
sigs = re.findall('(\w*)(\w*)\s*sig\s+(\w+)\s+(?:extends)?\s*(\w+)?\s*\{([^{]+)\}', lines)

classNames = [s[2] for s in sigs]
classes = []
for s in sigs:
    features = re.findall('(\w*)\s*:\s*(one|some|lone)?\s*(\w*)', s[4])
    c = {
        'name': s[2], 
        'isAbstract': 'abstract' in s[0:2],
        'extends': s[3],
        'features': [],
        'associations': []
     }
    for f in features:
        if(f[2] in classNames):
            c['associations'].append({
            'name': f[0],
            'arity': f[1],
            'target': f[2] 
            })
        else:
            c['features'].append({
                'name': f[0],
                'arity': f[1],
                'type': f[2] 
            })
    classes.append(c)
    
# cd output: classes + attributes
classesStr = ''

for c in classes:
    line = '\t{abstract}class {className}{extends}'.format(abstract='abstract ' if c['isAbstract'] else '', 
                                                           className=c['name'], 
                                                           extends = '' if (c['extends']=='') else (' extends '+c['extends']))
#     if not(c in fields.keys()):
#         classesStr+=(line+';\n')
#         continue
    fieldsTxt = ''
    for f in c['features']:
        fieldsTxt+= f' {f["type"]} {f["name"]}; '
        
    line+= f' {{{fieldsTxt}}}'

    classesStr+= (line+'\n')
    
# associations
associations = []

for c in classes:
    for a in c['associations']:
        a['source'] = c['name']
        a['arity2'] = '-'
        b = False
        for rel in associations:
            if rel['source']==a['target'] and a['source']==rel['target']:
                rel['type'] = '--'
                rel['arity2'] = a['arity']
                b = True
                break
        if not b:
            a['type'] = '->'
            associations.append(a)

            

def get_arity(argument):
    switcher = {
        '': '[0..*]',
        'some': '[1..*]',
        'one': '[1..1]',
        'lone': '[0..1]'
    }
    return switcher.get(argument, '')

associationsStr = ''
for a in associations:
#     association WorkPlace [1] Employee (of) -> (worksIn) Address [1..3];
    name = a['name']
    arity = get_arity(a['arity'])
    arity2 = get_arity(a['arity2'])
    src = a['source']
    tgt = a['target']
    atype = a['type']
    associationsStr += f'\tassociation {name} {src} {arity2} {atype} {tgt} {arity};\n'
output = f"""
package alloy2cd;

classdiagram {module} {{

{classesStr}

{associationsStr}
  
}}

"""

print(output)


package alloy2cd;

classdiagram alloy2cd {

	abstract class Vehicle {}
	class Car extends Vehicle { Bool isdeflated; }
	class Person { String name;  Int age; }
	class Wheel {}


	association wheels Car [1..1] -- Wheel [1..*];
	association cars Person  -> Car [1..*];

  
}


